In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/victorverboogen/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/victorverboogen/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/victorverboogen/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
df= pd.read_csv("Train_ML.csv")

In [4]:
df.head(5)

,ID,keywords,age,sex
0,1,fibre:16;quoi:1;dangers:1;combien:1;hightech:1...,62,F
1,2,restaurant:1;marrakech.shtml:1,35,M
2,3,payer:1;faq:1;taxe:1;habitation:1;macron:1;qui...,45,F
3,4,rigaud:3;laurent:3;photo:11;profile:8;photopro...,46,F
4,5,societe:1;disparition:1;proche:1;m%c3%a9lanie....,42,F


In [5]:
df.isnull().sum()

ID               0
keywords    846396
age              0
sex              0
dtype: int64

In [6]:
df.dropna(subset=['keywords'], inplace=True)

In [7]:
df.shape

(6418659, 4)

In [8]:
predict=df[0:10000].copy()

In [9]:
predict.shape

(10000, 4)

# Text Processing


In [10]:
from nltk.corpus import stopwords

In [11]:
def keyword_split_extract(keywords):
    
    result = ""
    
    raw_keywords = keywords.split(';')
    for unit in raw_keywords:
        keyword_freq = unit.split(':')
        keyword = keyword_freq[0].lower()
        try:
            if keyword not in stopwords.words('french'):
                result += f"{keyword} " * int(keyword_freq[1])
        except: pass
   
    return result.strip()

In [12]:
predict["keywords"]=predict["keywords"].apply(keyword_split_extract)

In [13]:
predict.head()

,ID,keywords,age,sex
0,1,fibre fibre fibre fibre fibre fibre fibre fibr...,62,F
1,2,restaurant marrakech.shtml,35,M
2,3,payer faq taxe habitation macron detail programme,45,F
3,4,rigaud rigaud rigaud laurent laurent laurent p...,46,F
4,5,societe disparition proche m%c3%a9lanie.gonide...,42,F


# Dummy


In [14]:
predict["sex"]=predict["sex"].replace("M",1)
predict["sex"]=predict["sex"].replace("F",0)

In [15]:
predict.head()

,ID,keywords,age,sex
0,1,fibre fibre fibre fibre fibre fibre fibre fibr...,62,0
1,2,restaurant marrakech.shtml,35,1
2,3,payer faq taxe habitation macron detail programme,45,0
3,4,rigaud rigaud rigaud laurent laurent laurent p...,46,0
4,5,societe disparition proche m%c3%a9lanie.gonide...,42,0


In [16]:
predict.dropna(subset=['keywords'], inplace=True)

In [17]:
predict.shape

(10000, 4)

In [18]:
predict.count()

ID          10000
keywords    10000
age         10000
sex         10000
dtype: int64

# Split


In [19]:
from sklearn.model_selection import train_test_split
Xtrain ,Xtest ,ytrain ,ytest = train_test_split(predict["keywords"],predict["sex"], test_size = 0.3, random_state = 123)

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
vc_tf_idf = TfidfVectorizer()

In [21]:
Xtrain.head

<bound method NDFrame.head of 3540     impossible suppression suppression+email+sur+i...
11259    cote condrieu guigal vallee rotie blanc rhone ...
8906     vue trois trois mort mort mort mort photos deu...
431      perigueux+24000 dordogne agence+immobiliere ag...
10709    equipe info info bleus ecran officiel crystal ...
                               ...                        
11095    comment protege protege ecriture ecriture foru...
8739     coups frappe lrem recettes recettes princesse ...
5838     forum carrosserie peinture medicaments quel vo...
1593     emmanuel programme livre macron revolution med...
4129     france france france france france france fran...
Name: keywords, Length: 7000, dtype: object>

In [22]:
vc_tf_idf.fit(Xtrain)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [23]:
Xtrain_tf = vc_tf_idf.transform(Xtrain)
Xtest_tf = vc_tf_idf.transform(Xtest)

In [28]:
#Random forest
from sklearn.ensemble import RandomForestClassifier as rdmfc

In [29]:
rfc = rdmfc(n_estimators = 50, random_state = 123)

In [30]:
rfc.fit(Xtrain_tf,ytrain)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=50,
                       n_jobs=None, oob_score=False, random_state=123,
                       verbose=0, warm_start=False)

In [31]:
prediction["RFC"] = rfc.predict(Xtest_tf)

In [32]:
accuracy_score(ytest,prediction["RFC"])

0.5913333333333334

In [33]:
print("rfc train score:", rfc.score(Xtrain_tf, ytrain))
print("rfc test score:", rfc.score(Xtest_tf, ytest))

rfc train score: 0.9174285714285715
rfc test score: 0.5913333333333334


# Predicting AGE

In [33]:
#linear regression
Xtrain,Xtest,ytrain_a,ytest_a = train_test_split(predict["keywords"],predict["age"], test_size = 0.3, random_state = 123)

In [34]:
from sklearn.linear_model import LinearRegression
LR = LinearRegression()

In [35]:
lm_reg = LR.fit(Xtrain_tf,ytrain_a) #training the algorithm
age_predict = LR.predict(Xtest_tf)

In [36]:
print(LR.intercept_)
print(LR.coef_)

45.498707124602376
[ 49.97003953 -94.47095476  48.63039007 ...  38.96159386  59.37764993
  -6.42518713]


In [37]:
from sklearn import metrics
print('Mean Absolute Error:', metrics.mean_absolute_error(ytest_a, age_predict))  
print('Mean Squared Error:', metrics.mean_squared_error(ytest_a, age_predict))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(ytest_a, age_predict)))

Mean Absolute Error: 43.14544134601952
Mean Squared Error: 7690.527933213911
Root Mean Squared Error: 87.69565515585086


In [38]:
#Random forest
from sklearn.ensemble import RandomForestRegressor as rdm

In [39]:
rfr = rdm(n_estimators = 50, random_state = 123)

In [40]:
rfr.fit(Xtrain_tf, ytrain_a)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=50,
                      n_jobs=None, oob_score=False, random_state=123, verbose=0,
                      warm_start=False)

In [41]:
prediction["RFR"] = rfr.predict(Xtest_tf)

In [42]:
errors = abs(prediction["RFR"] - ytest_a)
mape = 100 * (errors / ytest_a)
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

Accuracy: 73.85 %.


In [43]:
ytest_a.head(5)

2952     70
650      26
10802    46
457      30
4740     22
Name: age, dtype: int64

In [44]:
ytest_a.shape

(3000,)

# testing the model on the test dataset

In [46]:
df_test = pd.read_csv('Test_ML.csv')

In [47]:
df_test.head(5)

,ID,keywords,age,sex
0,1,NaN,NaN,NaN
1,2,cecilia.gosselin:1;flash:1;ville:1;obseques:1;...,NaN,NaN
2,3,p1_1697235:1;peut:1;jcms:1;les:1;acceptees:1;p...,NaN,NaN
3,4,002lundu83vnndv:1,NaN,NaN
4,5,high:3;patisserie:1;apple:3;tech:3;obseques:1;...,NaN,NaN


In [48]:
df_test.isnull().sum()

ID                0
keywords     363042
age         3111785
sex         3111785
dtype: int64

In [49]:
df_test.dropna(subset=['keywords'], inplace=True)
df_test.shape

(2748743, 4)

In [50]:
predict_test=df_test[0:1000].copy()

In [51]:
predict_test.head(5)

,ID,keywords,age,sex
1,2,cecilia.gosselin:1;flash:1;ville:1;obseques:1;...,NaN,NaN
2,3,p1_1697235:1;peut:1;jcms:1;les:1;acceptees:1;p...,NaN,NaN
3,4,002lundu83vnndv:1,NaN,NaN
4,5,high:3;patisserie:1;apple:3;tech:3;obseques:1;...,NaN,NaN
5,6,disparition:1;vue:1;maelys:1;deuxieme:1;place:...,NaN,NaN


In [52]:
predict_test["keywords"]= predict_test["keywords"].apply(keyword_split_extract)

In [53]:
predict_test.shape

(1000, 4)

In [54]:
predict_test.head()

,ID,keywords,age,sex
1,2,cecilia.gosselin flash ville obseques economie...,NaN,NaN
2,3,p1_1697235 peut jcms acceptees beneficiaire as...,NaN,NaN
3,4,002lundu83vnndv,NaN,NaN
4,5,high high high patisserie apple apple apple te...,NaN,NaN
5,6,disparition vue maelys deuxieme place actu fla...,NaN,NaN


In [55]:
predict_test.count()

ID          1000
keywords    1000
age            0
sex            0
dtype: int64

In [56]:
predict_test.dropna(subset=['keywords'], inplace=True)

In [57]:
predict_test.count()

ID          1000
keywords    1000
age            0
sex            0
dtype: int64

In [58]:
key_test = vc_tf_idf.transform(predict_test["keywords"])

In [59]:
prediction["gender"] = pd.DataFrame(nb.predict(key_test))
prediction["gender"]

,0
0,1
1,1
2,1
3,1
4,1
...,...
995,1
996,0
997,1
998,0


In [60]:
prediction['gender'].isnull().sum()
prediction['gender'].count()

0    1000
dtype: int64

In [61]:
prediction["age"] = pd.DataFrame(LR.predict(key_test))
prediction["age"]

,0
0,86.587631
1,47.786324
2,45.498707
3,70.098314
4,45.925770
...,...
995,219.788179
996,11.740454
997,109.742498
998,52.195409


In [62]:
predict_test['sex_pred']=prediction['gender']
predict_test['age_pred']=prediction['age']
predict_test.head(5)

,ID,keywords,age,sex,sex_pred,age_pred
1,2,cecilia.gosselin flash ville obseques economie...,NaN,NaN,1.0,47.786324
2,3,p1_1697235 peut jcms acceptees beneficiaire as...,NaN,NaN,1.0,45.498707
3,4,002lundu83vnndv,NaN,NaN,1.0,70.098314
4,5,high high high patisserie apple apple apple te...,NaN,NaN,1.0,45.925770
5,6,disparition vue maelys deuxieme place actu fla...,NaN,NaN,1.0,45.925770


In [63]:
prediction = predict_test[['ID','age_pred','sex_pred']].copy()
prediction["sex_pred"]=prediction["sex_pred"].replace(1,"M")
prediction["sex_pred"]=prediction["sex_pred"].replace(0,"F")
prediction.head(5)

,ID,age_pred,sex_pred
1,2,47.786324,M
2,3,45.498707,M
3,4,70.098314,M
4,5,45.925770,M
5,6,45.925770,M


In [64]:
prediction.isnull().sum()
prediction.count()

ID          1000
age_pred     870
sex_pred     870
dtype: int64

In [65]:
prediction.to_csv('age_gender_prediction.csv')